## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 📄 Azure AI Agent with File Search - Financial Documents 🏦

This notebook demonstrates how to create an Azure AI agent using `AzureAIProjectAgentProvider` with file search capabilities for document analysis, such as loan policies, compliance documents, and financial reports.

## Features Covered:
- File upload and management for financial documents
- Vector store creation and management
- File search tool configuration 
- Document-based Q&A for loan policies and compliance
- Resource cleanup and management

### ⚠️ Important Disclaimer ⚠️
> **This notebook demonstrates document search capabilities using sample financial documents. Always verify information with official sources and qualified professionals.**

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Microsoft Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with:
   - `AI_FOUNDRY_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`
4. **Dependencies**: Required agent-framework packages installed
5. **Sample Documents**: Financial documents (PDF/TXT) in `./resources` folder

If you need to use a different tenant:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries using the `AzureAIProjectAgentProvider` API:

In [ ]:
import os
from pathlib import Path

# Core agent framework imports
from agent_framework import HostedFileSearchTool, HostedVectorStoreContent
from agent_framework.azure import AzureAIProjectAgentProvider

# Azure SDK imports for file/vector store operations
from azure.ai.agents.aio import AgentsClient
from azure.ai.agents.models import FileInfo, VectorStore
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

# Load environment variables
notebook_path = Path().absolute()
load_dotenv('../../.env')

# Verify environment setup
endpoint = os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')
model = os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print("🔧 Environment Configuration:")
print(f"✅ Project Endpoint: {endpoint[:50]}..." if endpoint else "❌ AI_FOUNDRY_PROJECT_ENDPOINT not set")
print(f"✅ Model Deployment: {model}" if model else "❌ AZURE_AI_MODEL_DEPLOYMENT_NAME not set")

## Define Sample Queries 🏦

Let's define queries about loan policies and compliance:

In [ ]:
# Queries for loan policy and compliance documents
USER_INPUTS = [
    "What are the minimum credit score requirements for a mortgage loan?",
    "What documentation is required for a personal loan application?",
    "Can you summarize the key loan policies in the documents?",
    "What are the interest rate ranges mentioned for different loan types?",
]

## Main File Search Example 📄

This example demonstrates the complete workflow using the NEW `AzureAIProjectAgentProvider` API:
1. Upload a financial document (loan policies)
2. Create a vector store
3. Create a file search tool
4. Create an agent with file search capabilities
5. Query the agent about loan policies and compliance

In [ ]:
async def main() -> None:
    """Main function demonstrating Azure AI agent with file search for financial document analysis."""
    file: FileInfo | None = None
    vector_store: VectorStore | None = None

    # Step 1: Create sample financial document
    Path("./resources").mkdir(exist_ok=True)
    
    sample_content = """# CONTOSO BANK - LOAN POLICY DOCUMENT

## MORTGAGE LOANS
- Minimum Credit Score: 620
- Interest Rate Range: 6.0% - 7.5% APR
- Down Payment: Minimum 20% for conventional
- Required Documents: Income verification, tax returns (2 years), bank statements

## PERSONAL LOANS
- Minimum Credit Score: 650
- Interest Rate Range: 8.0% - 15.0% APR
- Loan Amount: $1,000 - $50,000
- Required Documents: ID, income proof, employment verification

## AUTO LOANS
- Minimum Credit Score: 600
- Interest Rate Range: 5.5% - 12.0% APR
- Maximum Loan-to-Value: 100% for new vehicles
- Required Documents: Vehicle information, insurance proof, income verification

## BUSINESS LOANS
- Minimum Credit Score: 680
- Interest Rate Range: 7.0% - 14.0% APR
- Required: Business plan, 2 years financial statements, personal guarantee
"""
    
    sample_file_path = Path("./resources") / "loan_policies.md"
    with open(sample_file_path, "w") as f:
        f.write(sample_content)
    print(f"✅ Created sample file: {sample_file_path}")

    async with (
        AzureCliCredential() as credential,
        AgentsClient(endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"], credential=credential) as agents_client,
        AzureAIProjectAgentProvider(credential=credential) as provider,
    ):
        try:
            # Step 2: Upload file and create vector store using AgentsClient
            print("📤 Uploading file to vector store...")
            file = await agents_client.files.upload_and_poll(
                file_path=str(sample_file_path), purpose="assistants"
            )
            print(f"✅ Uploaded file, file ID: {file.id}")

            vector_store = await agents_client.vector_stores.create_and_poll(
                file_ids=[file.id], name="loan_policies_vectorstore"
            )
            print(f"✅ Created vector store, vector store ID: {vector_store.id}")

            # Step 3: Create file search tool using agent-framework
            file_search_tool = HostedFileSearchTool(
                inputs=[HostedVectorStoreContent(vector_store_id=vector_store.id)]
            )
            
            # Step 4: Create agent using AzureAIProjectAgentProvider
            agent = await provider.create_agent(
                name="LoanPolicyAdvisor",
                instructions=(
                    "You are a helpful Loan Policy Advisor that can search through uploaded financial documents "
                    "to answer questions about loan policies, requirements, and compliance. "
                    "Always cite specific information from the documents and include relevant policy details."
                ),
                tools=file_search_tool,
            )
            
            # Step 5: Query the agent about loan policies
            print("\n=== 🏦 Querying Loan Policy Documents ===")
            for user_input in USER_INPUTS:
                print(f"\n🤔 Customer: '{user_input}'")
                response = await agent.run(user_input)
                print(f"📄 Advisor: {response.text}")

            print("\n=== ✅ File Search Example Complete ===")

        finally:
            # Step 6: Cleanup resources
            if vector_store:
                await agents_client.vector_stores.delete(vector_store.id)
                print(f"🗑️ Deleted vector store: {vector_store.id}")
            if file:
                await agents_client.files.delete(file.id)
                print(f"🗑️ Deleted file: {file.id}")

## Execute the Example 🚀

Run the main function to see the Loan Policy Advisor in action:

In [ ]:
# Run the main function
await main()

## Create a Sample Financial Policy File 📝

If you don't have a loan policy document, you can create one using this helper:

In [ ]:
def create_sample_financial_policy_file():
    """Create a sample loan policy file for testing."""
    sample_content = """
    CONTOSO BANK - COMPREHENSIVE LOAN POLICY DOCUMENT
    ==================================================
    Effective Date: January 1, 2024
    Version: 2.0
    
    SECTION 1: MORTGAGE LOANS
    -------------------------
    1.1 Eligibility Requirements:
        - Minimum Credit Score: 620 (Conventional), 580 (FHA)
        - Debt-to-Income Ratio: Maximum 43%
        - Employment History: Minimum 2 years
    
    1.2 Interest Rate Structure:
        - 30-Year Fixed: 6.25% - 7.50% APR
        - 15-Year Fixed: 5.75% - 6.75% APR
        - Adjustable Rate (5/1 ARM): 5.50% - 6.50% initial
    
    1.3 Required Documentation:
        - Government-issued ID
        - Pay stubs (last 30 days)
        - W-2 forms (last 2 years)
        - Tax returns (last 2 years)
        - Bank statements (last 2 months)
        - Proof of assets
    
    SECTION 2: PERSONAL LOANS
    -------------------------
    2.1 Eligibility Requirements:
        - Minimum Credit Score: 650
        - Minimum Annual Income: $25,000
        - Maximum Debt-to-Income: 40%
    
    2.2 Loan Terms:
        - Amount Range: $1,000 - $50,000
        - Interest Rate: 8.99% - 15.99% APR
        - Term Length: 12 - 60 months
    
    2.3 Required Documentation:
        - Government-issued ID
        - Proof of income
        - Proof of residence
    
    SECTION 3: AUTO LOANS
    ---------------------
    3.1 Eligibility Requirements:
        - Minimum Credit Score: 600
        - Down Payment: 10% recommended
    
    3.2 Loan Terms:
        - New Vehicles: 4.99% - 8.99% APR
        - Used Vehicles: 5.99% - 12.99% APR
        - Maximum Term: 72 months (new), 60 months (used)
    
    3.3 Required Documentation:
        - Valid driver's license
        - Proof of income
        - Vehicle information (VIN, mileage)
        - Proof of insurance
    
    SECTION 4: COMPLIANCE REQUIREMENTS
    ----------------------------------
    4.1 All loans must comply with:
        - Fair Lending Act
        - Truth in Lending Act (TILA)
        - Equal Credit Opportunity Act (ECOA)
    
    4.2 Documentation Retention:
        - Loan applications: 7 years
        - Denied applications: 25 months
        - Closed accounts: 7 years
    """
    
    # Save as text file
    Path("./resources").mkdir(exist_ok=True)
    with open("./resources/loan_policies.txt", "w") as f:
        f.write(sample_content)
    
    print("✅ Created sample loan_policies.txt in ./resources/")
    print("📝 Contains: Mortgage, Personal Loan, Auto Loan policies and compliance requirements")

# Uncomment to create sample file
# create_sample_financial_policy_file()

## Key Takeaways 📚

### Features

1. **File Upload**: Financial documents uploaded to Azure AI for searching
2. **Vector Stores**: Files organized in vector stores for efficient searching
3. **File Search Tool**: `HostedFileSearchTool` enables document-based Q&A
4. **Resource Management**: Track and cleanup uploaded files and vector stores

### Use Cases

- **Loan Policy Search**: Query loan requirements, rates, and eligibility
- **Compliance Documentation**: Search regulatory compliance documents
- **Financial Reports**: Analyze annual reports and financial statements
- **Customer Support**: Answer questions from product documentation

### Best Practices

1. **Document Organization**: Keep financial documents well-structured
2. **Error Handling**: Handle upload and processing errors gracefully
3. **Resource Cleanup**: Use try-finally blocks to track resources
4. **Document Quality**: Well-formatted documents yield better search results
5. **Query Clarity**: Frame financial questions clearly for accurate answers
